In [ ]:
!pip install fredapi

In [ ]:
!pip install python-dotenv

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt
from fredapi import Fred
import numpy as np

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["API_KEY"]

In [ ]:
fred = Fred(api_key=api_key) #replace with your API key here
data = fred.get_series("NROU")
data.tail()

In [ ]:
data = pd.DataFrame(data,columns=['Rate'])
data.reset_index(inplace=True)
data.rename(columns={"index": "Date"}, inplace=True)
data.head()

In [ ]:
data.loc[data['Date']=='1980-01-01']
data = data.iloc[124:]

In [ ]:
data.reset_index(drop=True)
data["Date"] = data["Date"].map(lambda x: x.toordinal())
data["Date"] -= data["Date"].min()
data["Date"] /= data["Date"].max()
data.head()

In [ ]:
rates_tensor = torch.from_numpy(data["Rate"].values).unsqueeze(dim=1)
dates_tensor = torch.from_numpy(data['Date'].values).unsqueeze(dim=1)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
cut_batch = int(len(rates_tensor) * .8)

X_train = dates_tensor[:cut_batch].type(torch.float32).to(device)
y_train = rates_tensor[:cut_batch].type(torch.float32).to(device)
X_test = dates_tensor[cut_batch:].type(torch.float32).to(device)
y_test = rates_tensor[cut_batch:].type(torch.float32).to(device)


In [ ]:
def plot_predictions(train_data = X_train, train_labels = y_train,
                     test_data = X_test, test_labels = y_test, predictions=None):
    plt.figure(figsize=(10,7))
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training Data")
    plt.scatter(test_data, test_labels, c='r', s=4, label="Testing data")
    if predictions is not None:
        plt.scatter(test_data, predictions, c='g', s=4, label="Predictions")
        
    plt.legend(prop={"size":12})

    plt.plot()


In [ ]:
plot_predictions()

In [ ]:
import torch.nn as nn
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1)

    def forward(self,x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

In [ ]:
torch.manual_seed(42)
model0 = LinearRegressionModel()
list(model0.parameters())

In [ ]:
model0.state_dict()

In [ ]:
loss_fn = nn.L1Loss()

optimizer = torch.optim.Adam(model0.parameters(), lr=.001)

In [ ]:
def noLossFr(epochs):
    for epoch in range(epochs):
        model0.train()
        x_pred = model0(X_train)
        loss = loss_fn(y_train, x_pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss)

noLossFr(10000)

In [ ]:
model0.eval()
with torch.inference_mode():
    test_preds = model0(X_test)
    loss = loss_fn(test_preds, y_test)
    plot_predictions(predictions = test_preds)
    print(loss)
